In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# function courtesy of CryptoDataDownload.com
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
                            
        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'cb_{pair_split[0] + pair_split[1]}_daily.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

In [3]:
fetch_daily_data('BTC/USD')

In [4]:
btc = pd.read_csv('cb_BTCUSD_daily.csv')

In [5]:
btc.head()

,unix,low,high,open,close,volume,date,vol_fiat
0,1610323200,30100.00,38273.88,38168.89,33224.28,81810.142934,2021-01-11,2.718083e+09
1,1610236800,34444.00,41452.12,40257.43,38171.57,43736.570316,2021-01-10,1.669494e+09
2,1610150400,38800.00,41406.94,40642.15,40257.43,27152.971029,2021-01-09,1.093109e+09
3,1610064000,36565.08,41986.37,39510.55,40665.15,48522.484903,2021-01-08,1.973174e+09
4,1609977600,36200.00,40425.00,36859.26,39505.56,50346.305691,2021-01-07,1.988959e+09


In [6]:
btc.drop('unix', axis=1, inplace=True)

In [8]:
btc['date'] = pd.to_datetime(btc['date'])

In [9]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   low       300 non-null    float64       
 1   high      300 non-null    float64       
 2   open      300 non-null    float64       
 3   close     300 non-null    float64       
 4   volume    300 non-null    float64       
 5   date      300 non-null    datetime64[ns]
 6   vol_fiat  300 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 16.5 KB


In [10]:
btc.set_index('date', inplace=True)

In [11]:
btc.head()

,low,high,open,close,volume,vol_fiat
date,,,,,,
2021-01-11,30100.00,38273.88,38168.89,33224.28,81810.142934,2.718083e+09
2021-01-10,34444.00,41452.12,40257.43,38171.57,43736.570316,1.669494e+09
2021-01-09,38800.00,41406.94,40642.15,40257.43,27152.971029,1.093109e+09
2021-01-08,36565.08,41986.37,39510.55,40665.15,48522.484903,1.973174e+09
2021-01-07,36200.00,40425.00,36859.26,39505.56,50346.305691,1.988959e+09
